### 📦 Imports & Environment Setup

This cell imports everything required to run the chatbot.

- `requests` is used to call the TypeGPT API.
- `time`, `copy`, and `warnings` support flow execution and safety.
- `typing` helps with clean function signatures.
- `userdata` reads secrets securely from Google Colab (no hard-coded API keys).

Nothing runs here yet. This cell only prepares the environment.


In [204]:
# ===== Imports =====
import time
import copy
import warnings
import requests
from typing import List, Dict, Any

from google.colab import userdata  # for secrets


### 🔑 TypeGPT API Configuration

This cell handles all communication with the TypeGPT API.

What happens here:
- The API base URL is defined.
- The API key is securely loaded from **Colab Secrets**.
- `call_typegpt()` sends conversation messages to the model.
- The function returns only the assistant’s final text response.

If the API fails, it raises an error so we can catch it later.


In [205]:
# ===== TypeGPT API =====
TYPEGPT_API_BASE = "https://api.typegpt.net/v1"
TYPEGPT_API_KEY = userdata.get("TYPEGPT_API_KEY")  # <-- stored in Colab Secrets

def call_typegpt(messages: List[Dict[str, str]], model="openai/gpt-oss-20b"):
    url = f"{TYPEGPT_API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {TYPEGPT_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": 0.7,
    }
    res = requests.post(url, json=payload, headers=headers, timeout=30)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]


### 🧠 Minimal PocketFlow Engine

This cell defines a tiny workflow engine inspired by PocketFlow.

Key ideas:
- Each **Node** performs one small task.
- Nodes are connected using `then()` to form a loop.
- The **Flow** object runs nodes one by one.
- Execution continues until a node returns `None`.

This keeps the chatbot logic modular and easy to extend.


In [206]:
# ===== PocketFlow Core =====
class Node:
    def __init__(self):
        self.next_node = None

    def then(self, node):
        self.next_node = node
        return node

    def run(self, shared):
        out = self.exec(shared)
        return out

    def exec(self, shared):
        raise NotImplementedError


class Flow:
    def __init__(self, start):
        self.start = start

    def run(self, shared):
        curr = copy.copy(self.start)
        while curr:
            curr = curr.run(shared)


### ⌨️ Input Node

This node:
- Reads user input from the terminal.
- Stops the chatbot if the user types `exit` or `quit`.
- Stores the user message in shared memory.

This is the entry point of every chatbot turn.


In [207]:
class InputNode(Node):
    def exec(self, shared):
        text = input("You: ").strip()
        if text.lower() in ("exit", "quit"):
            return None
        shared["last_user"] = text
        return self.next_node


### 🗃️ Memory Node

This node:
- Maintains the full conversation history.
- Appends each user message to the shared `store`.
- Uses OpenAI-style message format (`role`, `content`).

This stored history is later sent to the LLM.


In [208]:
class MemoryNode(Node):
    def exec(self, shared):
        store = shared.setdefault("store", [])
        store.append({"role": "user", "content": shared["last_user"]})
        return self.next_node


### 🤖 API Node (LLM Call)

This node:
- Sends the full conversation history to TypeGPT.
- Receives the assistant’s response.
- Handles API errors gracefully.
- Stores the assistant reply back into memory.

This is where the actual intelligence happens.


In [209]:
class APINode(Node):
    def exec(self, shared):
        try:
            reply = call_typegpt(shared["store"])
        except Exception as e:
            reply = f"[API error] {e}"
        shared["store"].append({"role": "assistant", "content": reply})
        shared["last_assistant"] = reply
        return self.next_node


### 🖥️ Display Node

This node:
- Prints the assistant’s response clearly to the console.
- Acts as the final step before looping back to input.

After displaying the message, control returns to the InputNode.


In [210]:
class DisplayNode(Node):
    def exec(self, shared):
        print("\nAssistant:", shared["last_assistant"], "\n")
        return self.next_node


### 🔗 Wiring the Chatbot Flow

This cell connects all nodes into a loop:

Input → Memory → API → Display → Input

Each message follows this exact path.
The loop continues until the user exits.


In [211]:
# ===== Build Flow =====
input_node = InputNode()
memory_node = MemoryNode()
api_node = APINode()
display_node = DisplayNode()

input_node.then(memory_node)\
          .then(api_node)\
          .then(display_node)\
          .then(input_node)

flow = Flow(input_node)


### ▶️ Run the Chatbot

This cell:
- Initializes shared memory.
- Adds a system prompt to guide behavior.
- Starts the flow execution.

Once this cell runs, the chatbot stays active until `exit` is typed.


In [ ]:
# ===== Run =====
shared = {"store": [
    {"role": "system", "content": "You are a helpful assistant. Answer concisely."}
]}

print("Chatbot ready. Type 'exit' to stop.\n")
flow.run(shared)


Chatbot ready. Type 'exit' to stop.

You: hi

Assistant: Hello! 👋 How can I help you today? 

You: what is pockeflow?

Assistant: [API error] HTTPSConnectionPool(host='api.typegpt.net', port=443): Read timed out. (read timeout=30) 

You: hi

Assistant: Hi there! 👋 How can I help you today? 

You: who ru?

Assistant: I’m ChatGPT, a large language model created by OpenAI. I’m here to answer questions, help with writing, brainstorm ideas, explain concepts, and chat about almost anything you’re curious about. Just let me know what you need! 

You: What is pocket flow?

Assistant: **Pocket Flow** can mean a few different things depending on the field you’re looking at. Below are the most common uses of the term:

| Context | What it is | Typical use |
|---------|------------|-------------|
| **Portable water‑flow meter** | A small, hand‑held device (often called a “pocket‑size” flow meter) that measures flow rate, pressure, and temperature in pipes. | Leak detection, water‑use monitoring, u